# Interpolate weather forecast data
https://open-meteo.com/en/docs#latitude=47.3667&longitude=8.55&current=&minutely_15=temperature_2m,relative_humidity_2m,apparent_temperature,precipitation,wind_speed_10m,is_day,shortwave_radiation&hourly=cloud_cover&forecast_days=14

In [197]:
# module imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# data import
min = pd.read_csv("minForecast.csv")
hour = pd.read_csv("hourlyForecast.csv")


hour['date'] = pd.to_datetime(hour['date']).dt.tz_convert("Europe/Berlin")
min['date'] = pd.to_datetime(min['date']).dt.tz_convert("Europe/Berlin")


,date,temperature_2m,relative_humidity_2m,apparent_temperature,precipitation,wind_speed_10m,is_day,shortwave_radiation
0,2024-10-11 02:00:00+02:00,11.4,88.0,10.23947,0.1,7.072878,0.0,0.0


In [199]:
lirp_min = min.set_index("date")
lirp_min = lirp_min.reindex(pd.date_range(start=min['date'].iloc[0],end=min['date'].iloc[-1], freq = pd.Timedelta(seconds=30)))
lirp_min = lirp_min.interpolate(method="index")

lirp_hour = hour.set_index("date")
lirp_hour = lirp_hour.reindex(pd.date_range(start=min['date'].iloc[0],end=min['date'].iloc[-1], freq = pd.Timedelta(seconds=30)))
lirp_hour = lirp_hour.interpolate(method="linear")

merged = pd.concat([lirp_min,lirp_hour], axis = 1)


# Add other features

In [222]:
merged['weekday'] = merged.index.weekday
merged['minutes_since_midnight'] = merged.index.hour * 60 +  merged.index.minute

merged['opened'] = 1
merged.loc[merged['minutes_since_midnight'] > 22 * 60,'opened'] = 0
merged.loc[merged['minutes_since_midnight'] < 6 * 60,'opened'] = 0
# reorder
merged = merged[['weekday', 'minutes_since_midnight', 'opened', 'temperature_2m',
       'relative_humidity_2m', 'apparent_temperature',
       'precipitation', 'cloud_cover', 'wind_speed_10m',
       'shortwave_radiation', 'is_day']]

merged.to_csv(f"../data/forecast/forecast-{merged.index[0].date().isoformat()}.csv")

'2024-10-11'